In [1]:
cd Desktop/lambda_projects/food_desert/CAMS_ZIP

/home/jm/Desktop/lambda_projects/food_desert/CAMS_ZIP


In [ ]:
'''
Dependencies:

- geopandas
- descartes
- geopy
- numba

'''

In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon

In [ ]:
'''
Data files used in this notebook:

CAMS_ZIPCODE_STREET_SPECIFIC.cpg  
CAMS_ZIPCODE_STREET_SPECIFIC.shp
CAMS_ZIPCODE_STREET_SPECIFIC.dbf  
CAMS_ZIPCODE_STREET_SPECIFIC.shp.xml
CAMS_ZIPCODE_STREET_SPECIFIC.prj  
CAMS_ZIPCODE_STREET_SPECIFIC.shx
CAMS_ZIPCODE_STREET_SPECIFIC.sbn
CAMS_ZIPCODE_STREET_SPECIFIC.sbx
Listing_of_Active_Businesses.csv

'''

In [4]:
# Loading the data files
polygon_file = 'CAMS_ZIPCODE_STREET_SPECIFIC.shp'
polygons = gpd.read_file(polygon_file)

In [5]:
polygons.head(2)

,OBJECTID_1,OBJECTID,FID_1,Name,SDE_STATE_,ORIG_FID,Distance,Zip_Num,Shape_area,Shape_len,geometry
0,1,105,0,90001,0.0,8499,0.0,90001,9.562251e+07,55969.055693,"POLYGON ((6483980.349832192 1818555.587574929,..."
1,2,194,1,90004,0.0,8502,0.0,90004,8.419310e+07,61463.722654,"POLYGON ((6473729.180045858 1852848.640051693,..."


In [6]:
converted = polygons.to_crs(epsg=4326)

In [7]:
converted.head(2)

,OBJECTID_1,OBJECTID,FID_1,Name,SDE_STATE_,ORIG_FID,Distance,Zip_Num,Shape_area,Shape_len,geometry
0,1,105,0,90001,0.0,8499,0.0,90001,9.562251e+07,55969.055693,POLYGON ((-118.2562720327559 33.98921307224975...
1,2,194,1,90004,0.0,8502,0.0,90004,8.419310e+07,61463.722654,POLYGON ((-118.2904153141149 34.08336856317362...


In [8]:
from geopandas import GeoDataFrame, GeoSeries

# For google maps, we should change lat <-> lon
def swap_xy(geom):
 
    def swap_xy_coords(coords):
        for x, y in coords:
            yield (y, x)

    ring = geom.exterior
    shell = type(ring)(list(swap_xy_coords(ring.coords)))
    holes = list(geom.interiors)
    for pos, ring in enumerate(holes):
        holes[pos] = type(ring)(list(swap_xy_coords(ring.coords)))
    return type(geom)(shell, holes)
  
def flip_invert(poly):
  # Change to geoseries
  poly_geo = GeoSeries(poly)

  # Rotate and Flip
  return poly_geo.rotate(90).scale(-1)

In [9]:
#call the xy_swap and flip functions to transform to correct Polygon:
converted['xy_swapped'] = converted.geometry.apply(swap_xy)
converted['xy_swapped_flipped'] = converted.xy_swapped.apply(flip_invert)

In [10]:
converted.head(2)

,OBJECTID_1,OBJECTID,FID_1,Name,SDE_STATE_,ORIG_FID,Distance,Zip_Num,Shape_area,Shape_len,geometry,xy_swapped,xy_swapped_flipped
0,1,105,0,90001,0.0,8499,0.0,90001,9.562251e+07,55969.055693,POLYGON ((-118.2562720327559 33.98921307224975...,POLYGON ((33.98921307224975 -118.2562720327559...,POLYGON ((33.96552428750395 -118.2325832480101...
1,2,194,1,90004,0.0,8502,0.0,90004,8.419310e+07,61463.722654,POLYGON ((-118.2904153141149 34.08336856317362...,POLYGON ((34.08336856317362 -118.2904153141149...,POLYGON ((34.09625428569836 -118.3033010366397...


In [11]:
#new df w/ only the columns we need:
zip_df = converted[['Zip_Num', 'xy_swapped_flipped']]
zip_df = zip_df.rename(columns={'Zip_Num':'zip_code', 'xy_swapped_flipped': 'polygon_coords'})

In [12]:
zip_df.head(2)

,zip_code,polygon_coords
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...


In [13]:
def gen_points_inside_polygon(polygon, miles=1):
    '''Takes a polygon and number of miles as input, creates a square grid around the polygon edges
    and iterates in default one mile chunks exhaustively acorss the grid, returns list of all the points
    generated'''
    
    #convert miles to degrees latitude/longitude 
    lat_increment = miles * 0.0145
    lon_increment = miles * 0.02 #this is specifically for L.A. (varies based on lat)
    
    #get the four corners of a grid of max and min lat/long values 
    min_lat, min_long, max_lat, max_long = polygon.bounds
    
    #iterate over the grid one unit at a time, append lats and lons to lists:
    longs, lats = [], []
    
    #Latitudes: start one half unit away from min_lat:
    lat = min_lat + (lat_increment / 2)
    while lat < max_lat:
        lats.append(lat)
        lat += lat_increment
    lats.append(max_lat)
    
    #repeat for longitude; conversion is diff, 1 mile = 0.0145 degrees
    #start one half unit above the min_lat
    lon = min_long + (lon_increment / 2)
    while lon < max_long:
        longs.append(lon)
        #increase .02 degrees (longitude conversion for Los Angeles)
        lon += lon_increment
    longs.append(max_long)

    #iterate thru all latitude and longitude points, instantiate shapely Point objects
    #and append them to a list:
    points = []
    for i in range(len(lats)):
        for j in range(len(longs)):
            points.append(Point(lats[i], longs[j]))
            
    #iterate over the points and throw out ones outside the polygon:
    points_inside_polygon = []
    for i in range(len(points)):
        if polygon.contains(points[i]):
            points_inside_polygon.append(points[i])
            
    #return the list of Shapely points which are bound inside the polygon        
    return points_inside_polygon

In [14]:
#call the function and make a new column in our zip_df that holds a lit of points inside the polygon
zip_df['test_points'] = zip_df.polygon_coords.apply(gen_points_inside_polygon)

In [15]:
zip_df.head(2)

,zip_code,polygon_coords,test_points
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...,"[POINT (33.96386274228264 -118.2522613507889),..."
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...,"[POINT (34.0553455342375 -118.3081234685644), ..."


In [16]:
#create new column that displays the num of points inside the polygon
zip_df['num_points'] = zip_df['test_points'].apply(len)

In [17]:
zip_df.head(2)

,zip_code,polygon_coords,test_points,num_points
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...,"[POINT (33.96386274228264 -118.2522613507889),...",2
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...,"[POINT (34.0553455342375 -118.3081234685644), ...",4


In [18]:
zip_df.num_points.sum()

3637

In [19]:
#Load the business data and clean up the df to our final list of markets/groceries
df_business = pd.read_csv('Listing_of_Active_Businesses.csv')

# Return a dataframe that contains market, grocery or food.
df_business_market = df_business[df_business['PRIMARY NAICS DESCRIPTION'].str.contains('market|grocery', flags=re.IGNORECASE, regex=True) == True]

# Return a dataframe that doesn't contain 'telephone|business|research'
df_business_market_clean = df_business_market[~df_business_market['PRIMARY NAICS DESCRIPTION'].str.contains('telephone|business|research') == True]

# Extract Latitude and put it in a column
df_business_market_clean['LATITUDE'] = df_business_market_clean['LOCATION'].str.extract('\((.*),').astype(float)

# Extract Longitude and put it in a column
df_business_market_clean['LONGITUDE'] = df_business_market_clean['LOCATION'].str.extract(',(.*)\)').astype(float)

# Remove coordinates outside of LA
df_business_market_clean = df_business_market_clean[(df_business_market_clean['LONGITUDE'] < -117.7) & (df_business_market_clean['LONGITUDE'] > -118.9)]

# Dataframe with no Nulls for location
markets = df_business_market_clean.dropna(subset=['LOCATION'])

/home/jm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/jm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
drop_cols = ['LOCATION ACCOUNT #', 'BUSINESS NAME', 'LOCATION DESCRIPTION', 'MAILING ADDRESS',
             'MAILING CITY', 'MAILING ZIP CODE', 'NAICS', 'COUNCIL DISTRICT', 'LOCATION START DATE',
             'LOCATION END DATE']

markets = markets.drop(columns=drop_cols).reset_index(drop=True)


In [21]:
#markets['LOCATION'] = markets['LOCATION'].apply(Point)
for i in range(len(markets)):
    lat = markets.at[i, 'LATITUDE']
    lon = markets.at[i, 'LONGITUDE']
    markets.at[i, "LOCATION"] = Point(lat, lon)

In [22]:
markets.head(2)

,DBA NAME,STREET ADDRESS,CITY,ZIP CODE,PRIMARY NAICS DESCRIPTION,LOCATION,LATITUDE,LONGITUDE
0,SILVER MOON MARKET,2501 E 4TH STREET,LOS ANGELES,90033-4418,Grocery & related products,POINT (34.0401 -118.2106),34.0401,-118.2106
1,VENUS MARKET,10117 AVALON BLVD,LOS ANGELES,90003-4817,Grocery stores (including supermarkets & conve...,POINT (33.9442 -118.2651),33.9442,-118.2651


In [24]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def travel(lat, lon, bearing=0, miles=0.5):
    import geopy
    import geopy.distance
    
    # Starting point
    start = geopy.Point(lat, lon)

    # Initialized with a default distance of 0.5 mi.
    end = geopy.distance.geodesic(miles=miles)

    # Destination method which takes the starting point and bearings(North=0).
    # Return coordinates from start to distance,d, traveled
    return end.destination(point=start, bearing=bearing)
       
def make_circle(lat, lon, miles=0.5):
    '''takes a coordinate of latitude, longitude and a distance in miles, creates a 360 sided polygon around the '''
    circle_points = []  
    for i in range(360):
        new_point = travel(lat, lon, bearing=i, miles=miles)
        new_point = new_point[0], new_point[1]
        circle_points.append(new_point)
    circle = Polygon(circle_points)
    return circle

def get_groceries_within_circle(df, circle):
    df_copy = df.copy()
    in_circle = []
    #iterate over every grocery in the df
    for i in range(len(df_copy)):
        point = df_copy.iloc[i]['LOCATION']
        if circle.contains(point):
            # Grocery within Circle
            in_circle.append(True)
        else:
            # NoGrocery within Circle
            in_circle.append(False)
    df_copy['in_circle'] = in_circle
    # Returns a data frame containing all grocery stores within the circle
    return df_copy[df_copy['in_circle'] == True]

In [31]:
test_points = zip_df.iloc[0].test_points
test_point = test_points[0]
lat, lon = test_point.x, test_point.y
test_circle = make_circle(lat, lon, miles=0.5)


In [32]:
res = get_groceries_within_circle(markets, test_circle)

In [33]:
res

,DBA NAME,STREET ADDRESS,CITY,ZIP CODE,PRIMARY NAICS DESCRIPTION,LOCATION,LATITUDE,LONGITUDE,in_circle
417,7-ELEVEN STORE 34535A,8600 S CENTRAL AVENUE,LOS ANGELES,90002-1124,Grocery stores (including supermarkets & conve...,POINT (33.9601 -118.2563),33.9601,-118.2563,True
462,LA FAMILIA 2014,7919 S CENTRAL AVENUE,LOS ANGELES,90001-3317,Grocery stores (including supermarkets & conve...,POINT (33.9672 -118.2564),33.9672,-118.2564,True
600,NaN,940 E 87TH STREET,LOS ANGELES,90002-1103,Fruit & vegetable markets,POINT (33.9592 -118.2572),33.9592,-118.2572,True
1150,LA FAMILIA MARKET,7915 S CENTRAL AVENUE,LOS ANGELES,90001-3317,Grocery stores (including supermarkets & conve...,POINT (33.9673 -118.2564),33.9673,-118.2564,True
1324,NaN,8628 S CENTRAL AVENUE,LOS ANGELES,90002-1112,Grocery & related products,POINT (33.9599 -118.2563),33.9599,-118.2563,True
2433,SAVE A LOT #775,8601 HOOPER AVENUE,LOS ANGELES,90002-1144,Grocery stores (including supermarkets & conve...,POINT (33.9599 -118.2546),33.9599,-118.2546,True


In [34]:
def search_for_deserts(list_of_points):
    '''takes a list of Shapely Points, iterates over them and checks if grocery is nearby, 
    return False if groceries are nearby, True if food desert'''
    flag = False
    desert_list = []
    for point in list_of_points:
        lat, lon = point.x, point.y
        circle = make_circle(lat, lon)
        result_df = get_groceries_within_circle(markets, circle)
        if len(result_df) == 0:
            desert_list.append((lat, lon))
            flag = True
    if flag:
        return desert_list
    else:
        return flag

In [35]:
list_of_points = zip_df.iloc[0].test_points
search_for_deserts(list_of_points)

False

In [36]:
subset = zip_df.iloc[:2]
subset

,zip_code,polygon_coords,test_points,num_points
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...,"[POINT (33.96386274228264 -118.2522613507889),...",2
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...,"[POINT (34.0553455342375 -118.3081234685644), ...",4


In [37]:
subset['desert_search'] = subset.test_points.apply(search_for_deserts)

/home/jm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
subset.head()

,zip_code,polygon_coords,test_points,num_points,desert_search
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...,"[POINT (33.96386274228264 -118.2522613507889),...",2,False
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...,"[POINT (34.0553455342375 -118.3081234685644), ...",4,False


In [190]:
len(zip_df)

358

In [39]:
first50 = zip_df.iloc[:50]
second50 = zip_df.iloc[50:100]
third50 = zip_df.iloc[100:150]
fourth50 =zip_df.iloc[150:200]
fifth50 = zip_df.iloc[200:250]
sixth50 = zip_df.iloc[250:300]
last58 = zip_df.iloc[300:]

In [194]:
first50['desert_search'] = first50.test_points.apply(search_for_deserts)
first50

,zip_code,polygon_coords,test_points,num_points,desert_search
0,90001,POLYGON ((33.96552428750395 -118.2325832480101...,"[POINT (33.96386274228264 -118.2522613507889),...",2,False
1,90004,POLYGON ((34.09625428569836 -118.3033010366397...,"[POINT (34.0553455342375 -118.3081234685644), ...",4,False
2,90006,"POLYGON ((34.0562936344686 -118.2846231702724,...","[POINT (34.03742267504161 -118.291367217405), ...",2,False
3,90007,POLYGON ((34.01640841554789 -118.2687503598564...,"[POINT (34.01527600193315 -118.2870569773083),...",2,False
4,90008,"POLYGON ((33.9881316350926 -118.337430015794, ...","[POINT (33.98480939440927 -118.3486081987702),...",4,False
5,90010,POLYGON ((34.05957157434112 -118.3177037934682...,[],0,False
6,90011,POLYGON ((34.02414408048078 -118.2651440991325...,"[POINT (34.00082048509941 -118.2703925536549),...",4,False
7,90012,"POLYGON ((34.0671096050594 -118.2161405903046,...","[POINT (34.0735077328263 -118.2572263594001), ...",2,False
8,90013,POLYGON ((34.03676464767061 -118.2345966056533...,[POINT (34.03979552022915 -118.2391841553905)],1,False
9,90014,POLYGON ((34.04006911077899 -118.2461140037061...,[],0,False


In [204]:
second50['desert_search'] = second50.test_points.apply(search_for_deserts)
second50

,zip_code,polygon_coords,test_points,num_points,desert_search
50,90230,POLYGON ((33.99021868933325 -118.3786740304815...,"[POINT (33.97958010218505 -118.4052351885026),...",6,"(False, [])"
51,90240,POLYGON ((33.95139663474716 -118.0997342193382...,"[POINT (33.93575214361699 -118.1077020311311),...",3,"(False, [])"
52,90241,POLYGON ((33.92105799306123 -118.1045283710712...,"[POINT (33.91006496135897 -118.1179046538797),...",4,"(False, [])"
53,90245,POLYGON ((33.91292305508428 -118.3853575775064...,"[POINT (33.89481588648329 -118.4049860284467),...",3,"(False, [])"
54,90247,POLYGON ((33.90753383163474 -118.2780184508395...,"[POINT (33.88730748567107 -118.3117847137752),...",3,"(False, [])"
55,90248,POLYGON ((33.88766503449096 -118.2539760453256...,"[POINT (33.86718439381167 -118.3090163084153),...",4,"(False, [])"
56,90263,POLYGON ((34.04509584703257 -118.7085916801656...,[],0,"(False, [])"
57,90272,POLYGON ((34.09334066142182 -118.4910629404864...,"[POINT (34.04393948890697 -118.5467905112189),...",21,"(False, [])"
58,90290,POLYGON ((34.12049403464077 -118.5633724526637...,"[POINT (34.05635250840596 -118.5902052069806),...",17,"(False, [])"
59,90291,POLYGON ((34.00293771702104 -118.4495559218664...,"[POINT (33.98159918287924 -118.4690123981757),...",2,"(False, [])"


In [40]:
third50['desert_search'] = third50.test_points.apply(search_for_deserts)
third50

/home/jm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,zip_code,polygon_coords,test_points,num_points,desert_search
100,90810,POLYGON ((33.81824250663111 -118.1924151916825...,"[POINT (33.80373707812628 -118.2019484844639),...",5,"[(33.803737078126275, -118.2019484844639), (33..."
101,90813,"POLYGON ((33.77169008024333 -118.185931204169,...","[POINT (33.75707007295394 -118.1972214660561),...",4,"[(33.75707007295394, -118.19722146605613), (33..."
102,90814,POLYGON ((33.74811521219225 -118.1383622440709...,[],0,False
103,90815,POLYGON ((33.83211896928319 -118.0989190739421...,"[POINT (33.76203248340066 -118.1051087988009),...",7,"[(33.76203248340066, -118.1051087988009), (33...."
104,90822,POLYGON ((33.78207963304754 -118.1218060346414...,[],0,False
105,90831,POLYGON ((33.76669551231173 -118.2008691787412...,[],0,False
106,90840,POLYGON ((33.78601071136687 -118.1076393704486...,[],0,False
107,91011,POLYGON ((34.21545227205264 -118.0790447848971...,"[POINT (34.15057190477002 -118.1483874050695),...",25,"[(34.15057190477002, -118.14838740506947), (34..."
108,91020,POLYGON ((34.20738201182864 -118.2219598652635...,[POINT (34.20940717672147 -118.2292811392743)],1,False
109,91023,"POLYGON ((34.23523806174995 -118.066537247847,...",[POINT (34.22088848895179 -118.0616737568463)],1,"[(34.220888488951786, -118.06167375684633)]"


In [ ]:
fourth50['desert_search'] = fourth50.test_points.apply(search_for_deserts)
fourth50

In [ ]:
fifth50['desert_search'] = fifth50.test_points.apply(search_for_deserts)
fifth50

In [ ]:
sixth50['desert_search'] = sixth50.test_points.apply(search_for_deserts)
sixth50

In [ ]:
last58['desert_search'] = last58.test_points.apply(search_for_deserts)
last58